In [1]:
# class Lambda(nn.Module):

#     def __init__(self, fn):
#         super(Lambda, self).__init__()
#         self.fn = fn

#     def forward(self, x):
#         return self.fn(x)

In [2]:
import learn2learn as l2l
for name in l2l.vision.benchmarks.list_tasksets():
    print(name)
    if("tiered-imagenet" == name):
        continue
    tasksets = l2l.vision.benchmarks.get_tasksets(name)

omniglot
Files already downloaded and verified
Files already downloaded and verified
mini-imagenet
tiered-imagenet
fc100
cifarfs


In [3]:
Datasets = [
    "omniglot",]
#             "mini-imagenet", 
#             "fc100", "cifarfs", "tiered-imagenet"]
# WaysAndShots = ["omniglot", "mini-imagenet", "fc100", "cifarfs", "tiered-imagenet"]
#!/usr/bin/env python3

"""
Demonstrates how to:
    * use the MAML wrapper for fast-adaptation,
    * use the benchmark interface to load Omniglot, and
    * sample tasks and split them in adaptation and evaluation sets.
"""

import random
import numpy as np
import torch
import learn2learn as l2l
import matplotlib.pyplot as plt
from torch import nn, optim
import os

In [ ]:
def accuracy(predictions, targets):
    predictions = predictions.argmax(dim=1).view(targets.shape)
    return (predictions == targets).sum().float() / targets.size(0)


def fast_adapt(batch, learner, loss, adaptation_steps, shots, ways, device):
    data, labels = batch
    data, labels = data.to(device), labels.to(device)

    # Separate data into adaptation/evalutation sets
    adaptation_indices = np.zeros(data.size(0), dtype=bool)
    adaptation_indices[np.arange(shots*ways) * 2] = True
    evaluation_indices = torch.from_numpy(~adaptation_indices)
    adaptation_indices = torch.from_numpy(adaptation_indices)
    adaptation_data, adaptation_labels = data[adaptation_indices], labels[adaptation_indices]
    evaluation_data, evaluation_labels = data[evaluation_indices], labels[evaluation_indices]

    # Adapt the model
    for step in range(adaptation_steps):
        train_error = loss(learner(adaptation_data), adaptation_labels)
        learner.adapt(train_error)

    # Evaluate the adapted model
    predictions = learner(evaluation_data)
    valid_error = loss(predictions, evaluation_labels)
    valid_accuracy = accuracy(predictions, evaluation_labels)
    return valid_error, valid_accuracy


def perform_experiment(
        dataset,
        ways=5,
        shots=1,
        meta_lr=0.003,
        fast_lr=0.5,
        meta_batch_size=32,
        adaptation_steps=1,
        num_iterations=64,#60000,
        cuda=True,
        seed=42,
        
):
    print("############### {0} ###############".format(dataset))
    Meta_Train_Accuracy = []
    Meta_Train_Error = []
    Meta_Val_Accuracy = []
    Meta_Val_Error = []
#     Iterations = []
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    device = torch.device('cpu')
    if cuda:
        torch.cuda.manual_seed(seed)
        device = torch.device('cuda')

    # Load train/validation/test tasksets using the benchmark interface
    tasksets = l2l.vision.benchmarks.get_tasksets(dataset,
                                                  train_ways=ways,
                                                  train_samples=2*shots,
                                                  test_ways=ways,
                                                  test_samples=2*shots,
                                                  num_tasks=20000,
                                                  root='~/data',
    )

    #     "omniglot",
#             "mini-imagenet", 
#             "fc100", "cifarfs", "tiered-imagenet"
    # Create model
    if(dataset == "omniglot"):
        model = l2l.vision.models.OmniglotFC(28 ** 2, ways)
    elif(dataset == "mini-imagenet"):
        model = l2l.vision.models.MiniImagenetCNN(ways)
    elif(dataset == "fc100"):
        features = l2l.vision.models.ConvBase(output_size=64, channels=3, max_pool=True)
        features = torch.nn.Sequential(features, Lambda(lambda x: x.view(-1, 256)))
        features.to(device)
        model = torch.nn.Linear(256, ways)
#     elif(dataset = ""):
    else:
        print("Error occured.")
        
    
    model.to(device)
    maml = l2l.algorithms.MAML(model, lr=fast_lr, first_order=False)
    opt = optim.Adam(maml.parameters(), meta_lr)
    loss = nn.CrossEntropyLoss(reduction='mean')

    for iteration in range(num_iterations):
        opt.zero_grad()
        meta_train_error = 0.0
        meta_train_accuracy = 0.0
        meta_valid_error = 0.0
        meta_valid_accuracy = 0.0
        for task in range(meta_batch_size):
            # Compute meta-training loss
            learner = maml.clone()
            batch = tasksets.train.sample()
            evaluation_error, evaluation_accuracy = fast_adapt(batch,
                                                               learner,
                                                               loss,
                                                               adaptation_steps,
                                                               shots,
                                                               ways,
                                                               device)
            evaluation_error.backward()
            meta_train_error += evaluation_error.item()
            meta_train_accuracy += evaluation_accuracy.item()

            # Compute meta-validation loss
            learner = maml.clone()
            batch = tasksets.validation.sample()
            evaluation_error, evaluation_accuracy = fast_adapt(batch,
                                                               learner,
                                                               loss,
                                                               adaptation_steps,
                                                               shots,
                                                               ways,
                                                               device)
            meta_valid_error += evaluation_error.item()
            meta_valid_accuracy += evaluation_accuracy.item()

        # Print some metrics
        
        
        meta_train_error =  meta_train_error / meta_batch_size
        meta_train_accuracy = meta_train_accuracy / meta_batch_size
        meta_val_error =  meta_valid_error / meta_batch_size
        meta_val_accuracy = meta_valid_accuracy / meta_batch_size
        if(iteration % 32 ==0):
            print('\n')
            print('Iteration', iteration)
            print('Meta Train Error', meta_train_error)
            print('Meta Train Accuracy', meta_train_accuracy)
            print('Meta Valid Error', meta_val_error)
            print('Meta Valid Accuracy', meta_val_accuracy)

        Meta_Train_Accuracy.append(meta_train_accuracy)
        Meta_Train_Error.append(meta_train_error)
        Meta_Val_Accuracy.append(meta_val_accuracy)
        Meta_Val_Error.append(meta_val_error)

        # Average the accumulated gradients and optimize
        for p in maml.parameters():
            p.grad.data.mul_(1.0 / meta_batch_size)
        opt.step()

    meta_test_error = 0.0
    meta_test_accuracy = 0.0
    for task in range(meta_batch_size):
        # Compute meta-testing loss
        learner = maml.clone()
        batch = tasksets.test.sample()
        evaluation_error, evaluation_accuracy = fast_adapt(batch,
                                                           learner,
                                                           loss,
                                                           adaptation_steps,
                                                           shots,
                                                           ways,
                                                           device)
        meta_test_error += evaluation_error.item()
        meta_test_accuracy += evaluation_accuracy.item()
    Meta_Test_Error = meta_test_error / meta_batch_size
    print('Meta Test Error', Meta_Test_Error)
    Meta_Test_Accuracy = meta_test_accuracy / meta_batch_size
    print('Meta Test Accuracy', Meta_Test_Accuracy)
    
    if not os.path.exists('plots'):
        os.makedirs('plots')
    if not os.path.exists('plots/acc'):
        os.makedirs('plots/acc')
    if not os.path.exists('plots/loss'):
        os.makedirs('plots/loss')
    ###### Plot Accuracies ######
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
    fig = plt.figure()
    ax = plt.subplot(111)
    ax.plot(list(range(0, len(Meta_Train_Accuracy))), Meta_Train_Accuracy, label="Meta Train ")
    ax.plot(list(range(0, len(Meta_Val_Accuracy))), Meta_Val_Accuracy, label="Meta Val")
#     ax.text((len(Meta_Val_Accuracy)/2), 0, 'Meta Test:{0}'.format(round(meta_test_accuracy, 2)), style='italic',
#         bbox={'facecolor': 'red', 'alpha': 0.25, 'pad': 5})
    # place a text box in upper left in axes coords
    ax.text(0.05, 0.5, 'Meta Test:{0}'.format(round(Meta_Test_Accuracy, 2)), transform=ax.transAxes, fontsize=14,
            verticalalignment='top', bbox=props)
    plt.title('Adaption')
    plt.xlabel('Iteration')
    plt.ylabel('Meta Accuracy')
    ax.legend()
    plt.savefig('./plots/acc/{0}_ways_{1}_shots_{2}_Acc_I_{3}.png'.format(dataset, ways, shots, iteration), 
                dpi=150)
    ###### Plot Accuracies ######

    ###### Plot Errors ######
    fig = plt.figure()
    ax = plt.subplot(111)
    ax.plot(list(range(0, len(Meta_Train_Error))), Meta_Train_Error, label="Meta Train ")
    ax.plot(list(range(0, len(Meta_Val_Error))), Meta_Val_Accuracy, label="Meta Val Lss")
#     ax.text((len(Meta_Val_Accuracy)/2),0, 'Meta Test Accuracy:{0}  Meta Test Loss:{1}'.format(round(meta_test_accuracy, 2),
#                               round(meta_test_error, 2)), style='italic',
#         bbox={'facecolor': 'red', 'alpha': 0.25, 'pad': 5})
    ax.text(0.05, 0.5, 'MTestAcc:{0}  MTestLoss:{1}'.format(round(Meta_Test_Accuracy, 2),
                                           round(Meta_Test_Error, 2)), transform=ax.transAxes, fontsize=14,
            verticalalignment='top', bbox=props)
    plt.title('Adaption')
    plt.xlabel('Iteration')
    plt.ylabel('Meta Loss')
    ax.legend()
    plt.savefig('./plots/loss/{0}_ways_{1}_shots_{2}_Loss_I_{3}.png'.format(dataset, ways, shots, iteration),
               dpi=150)
    ###### Plot Errors ######


if __name__ == '__main__':
    Ways = [5]
    Shots = [5, 1]
    Iterations = [10000, 60000, 120000]
    for ways in Ways:
        for shots in Shots:
            for iteration in Iterations:
                for dataset in Datasets:
                    perform_experiment(dataset=dataset,
                        ways=ways,
                        shots=shots,
                        meta_lr=0.003,
                        fast_lr=0.5,
                        meta_batch_size=32,
                        adaptation_steps=1,
                        num_iterations=iteration,
                        cuda=True,
                        seed=42)
    

############### omniglot ###############
Files already downloaded and verified
Files already downloaded and verified


Iteration 0
Meta Train Error 1.527568519115448
Meta Train Accuracy 0.332499987911433
Meta Valid Error 1.5436126105487347
Meta Valid Accuracy 0.34499999159015715


Iteration 32
Meta Train Error 0.7014891318976879
Meta Train Accuracy 0.7574999760836363
Meta Valid Error 0.6451591355726123
Meta Valid Accuracy 0.7824999745935202


Iteration 64
Meta Train Error 0.6121766334399581
Meta Train Accuracy 0.7862499784678221
Meta Valid Error 0.6224118201062083
Meta Valid Accuracy 0.7612499734386802


Iteration 96
Meta Train Error 0.43799832416698337
Meta Train Accuracy 0.8449999708682299
Meta Valid Error 0.49946191790513694
Meta Valid Accuracy 0.8199999779462814


Iteration 128
Meta Train Error 0.3983165379613638
Meta Train Accuracy 0.8649999815970659
Meta Valid Error 0.34178290190175176
Meta Valid Accuracy 0.8912499807775021


Iteration 160
Meta Train Error 0.32304693770129234
Met



Iteration 1568
Meta Train Error 0.07564363164419774
Meta Train Accuracy 0.9762499909847975
Meta Valid Error 0.10579123340721708
Meta Valid Accuracy 0.9649999849498272


Iteration 1600
Meta Train Error 0.07071453632670455
Meta Train Accuracy 0.9749999865889549
Meta Valid Error 0.11154137284756871
Meta Valid Accuracy 0.9612499866634607


Iteration 1632
Meta Train Error 0.06959389597250265
Meta Train Accuracy 0.9774999897927046
Meta Valid Error 0.15771416554343887
Meta Valid Accuracy 0.9499999806284904


Iteration 1664
Meta Train Error 0.06814345532984589
Meta Train Accuracy 0.9774999916553497
Meta Valid Error 0.18265888814858044
Meta Valid Accuracy 0.9437499772757292


Iteration 1696
Meta Train Error 0.08891816770483274
Meta Train Accuracy 0.9724999908357859
Meta Valid Error 0.1138288026995724
Meta Valid Accuracy 0.9637499861419201


Iteration 1728
Meta Train Error 0.08859225739070098
Meta Train Accuracy 0.9712499920278788
Meta Valid Error 0.16478035093314247
Meta Valid Accuracy 0.9512



Iteration 3136
Meta Train Error 0.05217837922191393
Meta Train Accuracy 0.9799999892711639
Meta Valid Error 0.165509357537303
Meta Valid Accuracy 0.9462499804794788


Iteration 3168
Meta Train Error 0.03295905891354778
Meta Train Accuracy 0.9924999978393316
Meta Valid Error 0.16651745287163067
Meta Valid Accuracy 0.951249985024333


Iteration 3200
Meta Train Error 0.0743671418640588
Meta Train Accuracy 0.9787499904632568
Meta Valid Error 0.157781228008389
Meta Valid Accuracy 0.9462499879300594


Iteration 3232
Meta Train Error 0.053041461706925475
Meta Train Accuracy 0.9787499886006117
Meta Valid Error 0.143116346320312
Meta Valid Accuracy 0.9624999891966581


Iteration 3264
Meta Train Error 0.05036575187477865
Meta Train Accuracy 0.9787499886006117
Meta Valid Error 0.16158326134427625
Meta Valid Accuracy 0.9549999851733446


Iteration 3296
Meta Train Error 0.059386855265984195
Meta Train Accuracy 0.9762499909847975
Meta Valid Error 0.1368865567346802
Meta Valid Accuracy 0.9587499853



Iteration 4704
Meta Train Error 0.05182285538103315
Meta Train Accuracy 0.9874999970197678
Meta Valid Error 0.09087021140931029
Meta Valid Accuracy 0.9724999871104956


Iteration 4736
Meta Train Error 0.03217266605497571
Meta Train Accuracy 0.9887499958276749
Meta Valid Error 0.12055278642583289
Meta Valid Accuracy 0.9662499893456697


Iteration 4768
Meta Train Error 0.047502306200840394
Meta Train Accuracy 0.9799999911338091
Meta Valid Error 0.13305138437499409
Meta Valid Accuracy 0.9549999833106995


Iteration 4800
Meta Train Error 0.053970689255947946
Meta Train Accuracy 0.9824999906122684
Meta Valid Error 0.15426523955829907
Meta Valid Accuracy 0.949999988079071


Iteration 4832
Meta Train Error 0.051968245686111914
Meta Train Accuracy 0.9824999943375587
Meta Valid Error 0.20765829843003303
Meta Valid Accuracy 0.9387499839067459


Iteration 4864
Meta Train Error 0.034587094775815785
Meta Train Accuracy 0.9912499953061342
Meta Valid Error 0.14432016893124455
Meta Valid Accuracy 0.



Iteration 6272
Meta Train Error 0.053366152248599974
Meta Train Accuracy 0.989999994635582
Meta Valid Error 0.1363384847427369
Meta Valid Accuracy 0.957499984651804


Iteration 6304
Meta Train Error 0.02849404917560605
Meta Train Accuracy 0.9937499966472387
Meta Valid Error 0.13815885120675375
Meta Valid Accuracy 0.962499987334013


Iteration 6336
Meta Train Error 0.03469999597973583
Meta Train Accuracy 0.989999994635582
Meta Valid Error 0.1580001238726254
Meta Valid Accuracy 0.952499981969595


Iteration 6368
Meta Train Error 0.04198712480683753
Meta Train Accuracy 0.9862499926239252
Meta Valid Error 0.1655906397209037
Meta Valid Accuracy 0.9574999827891588


Iteration 6400
Meta Train Error 0.031266418507584604
Meta Train Accuracy 0.9874999951571226
Meta Valid Error 0.10993437778597581
Meta Valid Accuracy 0.9662499874830246


Iteration 6432
Meta Train Error 0.029765690111162257
Meta Train Accuracy 0.989999994635582
Meta Valid Error 0.11220563794813643
Meta Valid Accuracy 0.963749988

...



Iteration 48032
Meta Train Error 0.00845339714557447
Meta Train Accuracy 0.9974999986588955
Meta Valid Error 0.1053171673138984
Meta Valid Accuracy 0.9762499909847975


Iteration 48064
Meta Train Error 0.020070346442015774
Meta Train Accuracy 0.989999994635582
Meta Valid Error 0.06636262735552556
Meta Valid Accuracy 0.9837499912828207


Iteration 48096
Meta Train Error 0.008983688316842153
Meta Train Accuracy 0.9974999986588955
Meta Valid Error 0.18200017016881986
Meta Valid Accuracy 0.9599999841302633


Iteration 48128
Meta Train Error 0.005973599303630728
Meta Train Accuracy 0.9987499993294477
Meta Valid Error 0.17251390756514695
Meta Valid Accuracy 0.9549999814480543


Iteration 48160
Meta Train Error 0.0024423522430638833
Meta Train Accuracy 1.0
Meta Valid Error 0.11671226036039606
Meta Valid Accuracy 0.9762499891221523


Iteration 48192
Meta Train Error 0.01433253856959027
Meta Train Accuracy 0.994999997317791
Meta Valid Error 0.1268886164107812
Meta Valid Accuracy 0.96749998442



Iteration 49632
Meta Train Error 0.011197593587120025
Meta Train Accuracy 0.9974999986588955
Meta Valid Error 0.18133112488817082
Meta Valid Accuracy 0.967499990016222


Iteration 49664
Meta Train Error 0.00047173082580975745
Meta Train Accuracy 1.0
Meta Valid Error 0.15936807499434735
Meta Valid Accuracy 0.9712499901652336


Iteration 49696
Meta Train Error 0.002861461948539201
Meta Train Accuracy 0.9987499993294477
Meta Valid Error 0.21337109605140125
Meta Valid Accuracy 0.9512499794363976


Iteration 49728
Meta Train Error 0.0056112509054884185
Meta Train Accuracy 0.9987499993294477
Meta Valid Error 0.23279752186999048
Meta Valid Accuracy 0.9649999868124723


Iteration 49760
Meta Train Error 0.0011283776838046933
Meta Train Accuracy 1.0
Meta Valid Error 0.1848960967067228
Meta Valid Accuracy 0.9587499834597111


Iteration 49792
Meta Train Error 0.0071267039694111745
Meta Train Accuracy 0.9974999986588955
Meta Valid Error 0.18183427675139452
Meta Valid Accuracy 0.9699999876320362





Iteration 51232
Meta Train Error 0.0015200087901963144
Meta Train Accuracy 0.9987499993294477
Meta Valid Error 0.1726562123978823
Meta Valid Accuracy 0.9649999868124723


Iteration 51264
Meta Train Error 0.001646943668037082
Meta Train Accuracy 1.0
Meta Valid Error 0.08872402080146902
Meta Valid Accuracy 0.984999991953373


Iteration 51296
Meta Train Error 0.011357950881095924
Meta Train Accuracy 0.9962499979883432
Meta Valid Error 0.18570573872272433
Meta Valid Accuracy 0.9637499880045652


Iteration 51328
Meta Train Error 0.0017892368042873308
Meta Train Accuracy 1.0
Meta Valid Error 0.18493333058470185
Meta Valid Accuracy 0.9612499810755253


Iteration 51360
Meta Train Error 0.0038654518912148994
Meta Train Accuracy 0.9987499993294477
Meta Valid Error 0.23781135122399966
Meta Valid Accuracy 0.9562499839812517


Iteration 51392
Meta Train Error 0.0048747990530175755
Meta Train Accuracy 0.9974999986588955
Meta Valid Error 0.36638816091389614
Meta Valid Accuracy 0.9437499791383743






Iteration 52800
Meta Train Error 0.0021596359650992714
Meta Train Accuracy 0.9987499993294477
Meta Valid Error 0.26278812488700964
Meta Valid Accuracy 0.9437499791383743


Iteration 52832
Meta Train Error 0.0037861393857521186
Meta Train Accuracy 0.9987499993294477
Meta Valid Error 0.11086898132851042
Meta Valid Accuracy 0.9787499904632568


Iteration 52864
Meta Train Error 0.0017395189515889697
Meta Train Accuracy 1.0
Meta Valid Error 0.07042208573535191
Meta Valid Accuracy 0.984999991953373


Iteration 52896
Meta Train Error 0.002555942254399479
Meta Train Accuracy 0.9987499993294477
Meta Valid Error 0.19251767880882387
Meta Valid Accuracy 0.9637499898672104


Iteration 52928
Meta Train Error 0.008289326058128044
Meta Train Accuracy 0.9962499979883432
Meta Valid Error 0.26967663777344675
Meta Valid Accuracy 0.962499987334013


Iteration 52960
Meta Train Error 0.01000695638741389
Meta Train Accuracy 0.9987499993294477
Meta Valid Error 0.2171957491237535
Meta Valid Accuracy 0.9599999



Iteration 54368
Meta Train Error 0.0036353915572426843
Meta Train Accuracy 0.9987499993294477
Meta Valid Error 0.17464508741504048
Meta Valid Accuracy 0.9674999881535769


Iteration 54400
Meta Train Error 0.007691072206194249
Meta Train Accuracy 0.9974999986588955
Meta Valid Error 0.10429640238543114
Meta Valid Accuracy 0.9662499874830246


Iteration 54432
Meta Train Error 0.0007344257792294417
Meta Train Accuracy 1.0
Meta Valid Error 0.18117353450912366
Meta Valid Accuracy 0.9637499842792749


Iteration 54464
Meta Train Error 0.0013243603532338
Meta Train Accuracy 1.0
Meta Valid Error 0.15301582728602448
Meta Valid Accuracy 0.962499987334013


Iteration 54496
Meta Train Error 0.00473670079523103
Meta Train Accuracy 0.9987499993294477
Meta Valid Error 0.11583497331929848
Meta Valid Accuracy 0.9699999894946814


Iteration 54528
Meta Train Error 0.01143994963448014
Meta Train Accuracy 0.9962499979883432
Meta Valid Error 0.3885186057749159
Meta Valid Accuracy 0.9362499825656414


Iterat

In [ ]:
# round(12.54673, 2)

In [ ]:


# fig = plt.figure()
# ax = plt.subplot()
# ax.plot(list(range(0, (10))), list(range(0, (10))), label="Meta Train")
# ax.text(5, 0, 'Meta Test:95%', style='italic',
#         bbox={'facecolor': 'red', 'alpha': 0.25, 'pad': 5}, ha='center', va='center')
# # ax.plot(list(range(0, len(Meta_Val_Accuracy))), Meta_Val_Accuracy, label="Meta Val")
# plt.title('Adaption')
# plt.xlabel('Iteration')
# plt.ylabel('Meta Accuracy')
# ax.legend()
# # plt.show()



# # plt.plot(list(range(0, len(Meta_Train_Accuracy))), Meta_Train_Accuracy,
# #         list(range(0, len(Meta_Val_Accuracy))), Meta_Val_Accuracy, label = ("A", "B"))#, list(range(0, len(Meta_Train_Error))), 
# #          Meta_Train_Error)#,label = ["Meta Train Accuracy", "Meta Train Error"])
# plt.savefig('foo.png')

In [ ]:
# plt.plot(list(range(0, len(Meta_Train_Accuracy))), Meta_Train_Accuracy,
#         list(range(0, len(Meta_Val_Accuracy))), Meta_Val_Accuracy, label = ("A", "B"))#, list(range(0, len(Meta_Train_Error))), 
# #          Meta_Train_Error)#,label = ["Meta Train Accuracy", "Meta Train Error"])